## Определения функций

In [ ]:
import sqlite3
import json
import pandas as pd
import numpy as np

conn = sqlite3.connect('M:\\sqlite3\\tweets.sqlite3')


def dict_generator(indict, pre=None):
    ''' Функция для обхода Nested structure внутри Json. Нужна только тогда когда структура 
            json-файла заранее неизвестна. для аргумента indict (строки json-файла) возвращает последовательность
            списков, где последний элемент - значение (лист дерева),а все остальные - ключи вложенных словарей'''
    pre = pre[:] if pre else []
    if isinstance(indict, dict):
        for key, value in indict.items():
            if isinstance(value, dict):
                for d in dict_generator(value, pre + [key]):
                    yield d
            elif isinstance(value, list) or isinstance(value, tuple):
                for v in value:
                    for d in dict_generator(v, pre +[key] ):
                        yield d
            else:
                yield pre + [key, value]
    else:
        yield pre+[indict]
        
        
def dict_to_df(dictname):
    ''' Перевод словаря из Json в Data Frame и дублирование строк для создания денормализованной таблицы'''
    def find_key_with_list(dictname):
        for i in dictname:
            if isinstance(dictname[i],list) or isinstance(dictname[i],tuple):
                return i
            
    def make_lists_in_dict(dictname):
        a=dictname.copy()
        for j in a:
            a[j]=[a[j]]
        return a
    
    
    
    def dict_to_table_structure(dictname,key_withlist):
        #shalow copy вводного словаря, чтобы не изменять значение внутри функции
        a=dictname.copy()
     
        if len(a[key_withlist])>1:

            for j in [j for j in a if j!=key_withlist]:
                a[j]=[a[j]]*len(a[key_withlist])
            return a
        elif len(a[key_withlist])==1:
            a[key_withlist]=a[key_withlist][0]
            a=make_lists_in_dict(a)
            return a
        elif len(a[key_withlist])==0:
            a[key_withlist]=np.nan
            a=make_lists_in_dict(a)
            return a
        

    
    return pd.DataFrame(dict_to_table_structure(dictname,find_key_with_list(dictname)))

## Загрузка в DataFrame

In [ ]:
df=pd.DataFrame({'name':[], 'text':[], 'country_code':[], 'display_url':[], 'lang':[], 'created_at':[], 'location':[]})
filename=u'M:\\sqlite3\\three_minutes_tweets.json.txt'
with open(filename,'r') as tweetFile:
    for it,line in enumerate([line for line in tweetFile]):
        #построчно обрабатываем файл с твитами
        tweet=json.loads(line)
        # создание целевого словаря, где на один твит приходится по одной записи для всех ключей, кроме 'display_url'
        tweet_record=dict.fromkeys(['name', 'text', 'country_code', 'display_url', 'lang', 'created_at', 'location'])
        tweet_record['display_url']=[]
        
        # отбрасываем записи с ключом delete
        if not tweet.keys()==[u'delete']:
            for a in dict_generator(tweet):
            
#                 if a is None:
#                     pass
            
                if len(a)>=2:
                    if a[-2] in tweet_record:

                        if len(a)==3 and a[-2]==u'name':
                            tweet_record['name']=a[-1]

                        elif len(a)==2 and a[-2]==u'text':
                            tweet_record['text']=a[-1]
                        elif a[-2]==u'display_url':
                            tweet_record[u'display_url'].append(a[-1])
                            
                        elif a[-2] in [u'country_code',u'lang',u'created_at',u'location']:
                            tweet_record[a[-2]]=a[-1]
                            
        tweet_record[u'display_url']=list(set(tweet_record[u'display_url']))
        
        #Заменить на: df.to_sql("Tweet", conn, if_exists="append")
        df=df.append(dict_to_df(tweet_record),ignore_index=True)

## Тест для загруженного DataFrame

In [ ]:
def picklines(thefile, whatlines):
    return [x for i, x in enumerate(thefile) if i in whatlines]

cnt=0
for i in range(1000):
    with open(filename,'r') as tweetFile:
        line=picklines(tweetFile,[i])[0]


        try:
            line=json.loads(line)
            if not all(df.loc[(df.name==line[u'user'][u'name']) & (df.text==line[u'text'])].created_at==line[u'created_at']):
                print 'ERROR!!',i,line
        except KeyError:
            print i,line
            cnt+=1
        except TypeError:
            print i,line
            cnt+=1

## Загрузка в базу данных

In [ ]:
c = conn.cursor()
df=df.rename(columns={'text':'tweet_text'})

df.to_sql("Tweet", conn, if_exists="replace")


In [ ]:
# Для нормализации БД создаются отдельные таблицы для Твитов и для ссылок в Твитах (вторая нормальная форма)
c.execute('''DROP TABLE Tweet_Table ; ''')
c.execute('''DROP TABLE Url_Table ; ''')

c.execute('''CREATE TABLE Tweet_Table
                (
                tweet_id INTEGER PRIMARY KEY AUTOINCREMENT,
               created_at,name,
               tweet_text,
               country_code,
               lang,
               location,
               tweet_sentiment
               );
                  ''')

c.execute('''INSERT INTO Tweet_Table (created_at,name,tweet_text,country_code,lang,location,tweet_sentiment)
                SELECT DISTINCT created_at,name,tweet_text,country_code,lang,location,tweet_sentiment FROM Tweet; ''')

c.execute('''CREATE TABLE Url_Table
                (
                tweet_id,
                display_url
               );
          ''')

c.execute(''' INSERT INTO Url_Table (tweet_id,display_url)
              SELECT t1.tweet_id,t2.display_url
              FROM Tweet_Table t1
              LEFT JOIN Tweet t2 ON t1.created_at=t2.created_at AND t1.name=t2.name AND t1.tweet_text=t2.tweet_text
              WHERE t2.display_url<>"None"

            ''')

In [ ]:

# подсчет настроений через поиск по словарю

# tweets_for_analysis=df.loc[(df.lang=='en')].tweet_text



dict_of_sentiment={}

with open(u'M:\\sqlite3\\AFINN-111.txt') as f:
    for line in f:
        dict_of_sentiment[line.strip().split('	')[0]]=int(line.strip().split('	')[1])
        

        
tweets_for_analysis=pd.read_sql('''SELECT tweet_id,tweet_text FROM Tweet_Table WHERE tweet_id IS NOT NULL;''',conn)


# для каждого твита считается настроение и загружается в таблицу по одной записи

for tweet_id,tweet in zip(tweets_for_analysis.tweet_id.values,tweets_for_analysis.tweet_text.values):
    if not tweet is None: 
        # нижний регистр, очистка от пробелов и разделение по словам
        bag_of_words=map(lambda x:x.lower(),tweet.strip().split(' '))
        cnt=0
        for i in bag_of_words:
            if i in dict_of_sentiment:
                cnt+=dict_of_sentiment[i]
        # загружаем в БД только твиты с ненулевым настроением
        if cnt!=0:
            to_append=pd.DataFrame({'tweet_id':[tweet_id],'tweet_sentiment':[cnt]});
            to_append.to_sql("Tweet_sentiment2", conn, if_exists="append",index=False)

In [ ]:
pd.read_sql('''SELECT tweet_id,tweet_text FROM Tweet_Table''',conn,chunksize=1000)

In [ ]:
# Найти страну-локацию-имя с максимальным и минимальным значением
def most_group(field,x):
    '''field - поле по которому данные группируются для подсчета минимального/максимального настроения 
        x - параметр принимающий значение 'MAX' или 'MIN'  '''
        return pd.read_sql(''' 

                        SELECT '''+field+'''

                        FROM
                        (
                        SELECT t2.'''+field+''',SUM(t1.tweet_sentiment) AS sentiment 
                        FROM Tweet_sentiment2 t1
                        LEFT JOIN Tweet_Table t2 ON t1.tweet_id=t2.tweet_id
                        GROUP BY '''+field+'''
                        ) t1
                        WHERE sentiment=(SELECT '''+x+ '''(sentiment) FROM 
                                                        (
                                                        SELECT t2.'''+field+''',SUM(t1.tweet_sentiment) AS sentiment 
                                                        FROM Tweet_sentiment2 t1
                                                        LEFT JOIN Tweet_Table t2 ON t1.tweet_id=t2.tweet_id
                                                        WHERE '''+field+''' IS NOT NULL AND
                                                        '''+field+'''<>''
                                                        GROUP BY '''+field+'''
                                                        ) t1
                                        )



                        ''',conn)[field].values[0]

In [ ]:
print 'Самая счастливая страна',most_group('country_code','MAX')
print 'Самая несчастная страна',most_group('country_code','MIN')

print 'Самая счастливая локация',most_group('location','MAX')
print 'Самая несчастная локация',most_group('location','MIN')

print 'Самый счастливый пользователь',most_group('name','MAX')
print 'Самый несчастный пользователь',most_group('name','MIN')

In [ ]:
pd.read_sql(''' SELECT location,sentiment

                        FROM
                        (
                        SELECT t2.location,SUM(t1.tweet_sentiment) AS sentiment 
                        FROM Tweet_sentiment2 t1
                        LEFT JOIN Tweet_Table t2 ON t1.tweet_id=t2.tweet_id
                        GROUP BY location ) t1
                        ORDER BY sentiment DESC''',conn)

In [ ]:
c.execute('''DROP TABLE Tweet_sentiment2;''')